In [1]:
import torch
import os, sys
import numpy as np
import shutil

In [2]:
HOME_DIRECTORY=os.path.abspath(os.path.join(os.getcwd(), os.pardir))

os.chdir(HOME_DIRECTORY)

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # sync ids with nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
os.environ["MKL_SERVICE_FORCE_INTEL"]="1"

In [4]:
# script params
port=5013
sampling_fn="random"
lSet_partition=1
base_seed=1
num_GPU=1
al_iterations=4
num_aml_trials=5 #50
budget_size=5000 #2500

dataset="CIFAR10"
init_partition=10
step_partition=10
clf_epochs=5 #150
num_classes=10

log_iter=40

#Data arguments
train_dir=f"{HOME_DIRECTORY}/data/{dataset}/train-{dataset}/"
test_dir=f"{HOME_DIRECTORY}/data/{dataset}/test-{dataset}/"
lSetPath=f"{HOME_DIRECTORY}/data/{dataset}/partition_{lSet_partition}/lSet_{dataset}.npy"
uSetPath=f"{HOME_DIRECTORY}/data/{dataset}/partition_{lSet_partition}/uSet_{dataset}.npy"
valSetPath=f"{HOME_DIRECTORY}/data/{dataset}/partition_{lSet_partition}/valSet_{dataset}.npy"

out_dir=f"{HOME_DIRECTORY}/sample_results_aml_transfer"

model_style="vgg_style"
model_type="vgg"
model_depth=16

## Using active sets found using VGG16 for finetuning Resnet18. 

- We need to perform following steps to run the transfer experiment. 
- These steps are important as we wanted each results directory to contains results for single experimental setting. 
- With this we avoid any unintentional errors which can be caused by saving/overwriting models, logs, checkpoints of more than one experiment in same results folder.
- Inorder to run transferability.
    * Step1: Locate the path of active sets from source network. In our case it is <current_directory>/sample_results_aml
    * Step2: To alleviate any bias in hyper-parameters selection when training the target architecture we again make use of AutoML running a random search over 50 trials. So in our new results directory (where we copy the active sets sampled using source architecture) we only have "best_automl_results" directory.
    * Step3: Incase any checkpoints exist in results directory, make sure to remove them.
   

In [5]:
#step1
activeset_dir=f"{HOME_DIRECTORY}/sample_results_aml"
!mkdir -p $out_dir
!scp -r $activeset_dir/best_automl_results $out_dir/

In [6]:
#step2: if there exist checkpoints remove them
dirlist = []
extraFiles = []
for root, dirs, files in os.walk(out_dir+"/"):
    for d in dirs:
        #append the dir name to the list
        if os.path.basename(d)=="checkpoints":
            dirlist.append(os.path.join(root,d))
    for f in files:
        #append the dir name to the list
        if os.path.splitext(f)[1] not in [".npy"]:
            extraFiles.append(os.path.join(root,f))

dirlist.sort()            
extraFiles.sort()            
    
#print all the dir names
for d in dirlist:
    print("Found", d)
    
    
# one can remove extrafiles too - however as they don't effect model learning - we omit to delete them in this notebook


Found /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/checkpoints
Found /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/20.0/random/vgg_depth_16/vanilla/checkpoints
Found /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/30.0/random/vgg_depth_16/vanilla/checkpoints
Found /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/40.0/random/vgg_depth_16/vanilla/checkpoints


In [7]:
# We don't want to remove checkpoints for initial lab data, so we remove rest.
best_base_path=None
for i,d in enumerate(dirlist):
    if i==0:
        best_base_path=d
        continue
    if os.path.exists(d):
        shutil.rmtree(d+"/")
        print("Removed ", d)

Removed  /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/20.0/random/vgg_depth_16/vanilla/checkpoints
Removed  /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/30.0/random/vgg_depth_16/vanilla/checkpoints
Removed  /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/40.0/random/vgg_depth_16/vanilla/checkpoints


In [8]:
# Now we are ready to run the transfer experiment except to setup automl results for base 10% model. 
# Our codebase assumes that we have run automl even for initial model.

automl_base_path=f"{out_dir}/auto_ml_results/lSet_{lSet_partition}/start_{base_seed}/{dataset}/10.0/vgg_depth_16/vanilla/trial-0/" 
!mkdir -p $automl_base_path
!scp -r $best_base_path $automl_base_path 


In [9]:
!python3 $HOME_DIRECTORY/tools/main_aml.py --n_GPU $num_GPU \
--port $port --sampling_fn $sampling_fn --lSet_partition $lSet_partition \
--seed_id $base_seed \
--init_partition $init_partition --step_partition $step_partition \
--dataset $dataset --budget_size $budget_size \
--out_dir $out_dir \
--num_aml_trials $num_aml_trials --num_classes $num_classes \
--al_max_iter $al_iterations \
--model_type $model_type --model_depth $model_depth \
--clf_epochs $clf_epochs \
--eval_period 1 --checkpoint_period 1 \
--lSetPath $lSetPath --uSetPath $uSetPath --valSetPath $valSetPath \
--train_dir $train_dir --test_dir $test_dir \
--dropout_iterations 25 \
--cfg configs/$dataset/$model_style/$model_type/R-18_4gpu_unreg.yaml \
--vaal_z_dim 32 --vaal_vae_bs 64 --vaal_epochs 2 \
--vaal_vae_lr 5e-4 --vaal_disc_lr 5e-4 --vaal_beta 1.0 --vaal_adv_param 1.0 \
--isTransferExp --transfer_model_type resnet_2 \
--transfer_model_style resnet_style --transfer_model_depth 18 --transfer_dir_specific vanilla \



========= [NO ADVANCED REGULARIZATION TRICK ACTIVATED] =========
~~~ out_dir:  /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/auto_ml_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla
check_aml_path: 
/nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/checkpoints
Auto ml already exists; So skip doing automl for this!
Number of trials found at /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/auto_ml_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla: 1
best_model_path chosen: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/auto_ml_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/trial-0/checkpoints/vlBest_acc_47.3_model_epoch_0006.pyth
===========In custom dum cfg===========
temp_cfg.OUT_DIR: /nfs/use

Files already downloaded and verified
Number of testing datapoints: 10000
Len(train_loader): 625
[train_al.py: 450]: Start epoch: 1
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "10/625", "loss": 2.487498044968, "lr": 0.000282441566, "top1_err": 81.250000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "20/625", "loss": 2.102945446968, "lr": 0.000282441566, "top1_err": 78.125000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "30/625", "loss": 1.986389040947, "lr": 0.000282441566, "top1_err": 71.875000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "40/625", "loss": 1.936379492283, "lr": 0.000282441566, "top1_err": 78.125000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "50/625", "loss": 2.043925523758, "lr": 0.000282441566, "top1_err": 71.875000000000}
[logging.py:  75]: json_stats: {"_type": "train_

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "500/625", "loss": 1.339190423489, "lr": 0.000282441566, "top1_err": 50.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "510/625", "loss": 1.618017554283, "lr": 0.000282441566, "top1_err": 56.250000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "520/625", "loss": 1.467140734196, "lr": 0.000282441566, "top1_err": 59.375000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "530/625", "loss": 1.447299182415, "lr": 0.000282441566, "top1_err": 50.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "540/625", "loss": 1.342341840267, "lr": 0.000282441566, "top1_err": 46.875000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "1/5", "iter": "550/625", "loss": 1.296297430992, "lr": 0.000282441566, "top1_err": 50.000000000000}
[logging.p

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "2/5", "iter": "340/625", "loss": 1.400408208370, "lr": 0.000282441566, "top1_err": 50.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "2/5", "iter": "350/625", "loss": 1.240910708904, "lr": 0.000282441566, "top1_err": 50.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "2/5", "iter": "360/625", "loss": 1.269426167011, "lr": 0.000282441566, "top1_err": 50.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "2/5", "iter": "370/625", "loss": 1.296802818775, "lr": 0.000282441566, "top1_err": 53.125000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "2/5", "iter": "380/625", "loss": 1.515165984631, "lr": 0.000282441566, "top1_err": 50.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "2/5", "iter": "390/625", "loss": 1.162063360214, "lr": 0.000282441566, "top1_err": 37.500000000000}
[logging.p

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "180/625", "loss": 1.021135628223, "lr": 0.000282441566, "top1_err": 34.375000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "190/625", "loss": 1.107691764832, "lr": 0.000282441566, "top1_err": 46.875000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "200/625", "loss": 1.009833931923, "lr": 0.000282441566, "top1_err": 37.500000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "210/625", "loss": 1.041412889957, "lr": 0.000282441566, "top1_err": 34.375000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "220/625", "loss": 1.200326144695, "lr": 0.000282441566, "top1_err": 37.500000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "230/625", "loss": 1.193438708782, "lr": 0.000282441566, "top1_err": 37.500000000000}
[logging.p

...

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "80/625", "loss": 1.610962927341, "lr": 0.000056882718, "top1_err": 59.375000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "90/625", "loss": 1.631058633327, "lr": 0.000056882718, "top1_err": 57.812500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "100/625", "loss": 1.604886472225, "lr": 0.000056882718, "top1_err": 59.375000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "110/625", "loss": 1.744554102421, "lr": 0.000056882718, "top1_err": 65.625000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "120/625", "loss": 1.604541182518, "lr": 0.000056882718, "top1_err": 60.937500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "130/625", "loss": 1.559441030025, "lr": 0.000056882718, "top1_err": 56.250000000000}
[logging.py:

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "580/625", "loss": 1.459370911121, "lr": 0.000056882718, "top1_err": 51.562500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "590/625", "loss": 1.566969990730, "lr": 0.000056882718, "top1_err": 59.375000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "600/625", "loss": 1.516179144382, "lr": 0.000056882718, "top1_err": 57.812500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "610/625", "loss": 1.520144641399, "lr": 0.000056882718, "top1_err": 54.687500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "3/5", "iter": "620/625", "loss": 1.516049444675, "lr": 0.000056882718, "top1_err": 53.125000000000}
[logging.py:  75]: json_stats: {"_type": "train_epoch", "epoch": "3/5", "loss": 1.542326683044, "lr": 0.000056882718, "top1_err": 56.490000000000}
[logging.py:  75]: json_stat

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "420/625", "loss": 1.457606375217, "lr": 0.000056882718, "top1_err": 51.562500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "430/625", "loss": 1.368809044361, "lr": 0.000056882718, "top1_err": 48.437500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "440/625", "loss": 1.402079820633, "lr": 0.000056882718, "top1_err": 53.125000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "450/625", "loss": 1.394842624664, "lr": 0.000056882718, "top1_err": 50.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "460/625", "loss": 1.438446462154, "lr": 0.000056882718, "top1_err": 50.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "4/5", "iter": "470/625", "loss": 1.422193288803, "lr": 0.000056882718, "top1_err": 51.562500000000}
[logging.p

[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "260/625", "loss": 1.440769791603, "lr": 0.000056882718, "top1_err": 46.875000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "270/625", "loss": 1.462140381336, "lr": 0.000056882718, "top1_err": 53.125000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "280/625", "loss": 1.447188615799, "lr": 0.000056882718, "top1_err": 54.687500000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "290/625", "loss": 1.436099648476, "lr": 0.000056882718, "top1_err": 50.000000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "300/625", "loss": 1.390639781952, "lr": 0.000056882718, "top1_err": 53.125000000000}
[logging.py:  75]: json_stats: {"_type": "train_iter", "epoch": "5/5", "iter": "310/625", "loss": 1.278931379318, "lr": 0.000056882718, "top1_err": 45.312500000000}
[logging.p

Model checkpoint saved at path: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/auto_ml_results/lSet_1/start_1/CIFAR10/40.0/random/vgg_depth_16/vanilla/trial-4/checkpoints/vlBest_acc_50.979999694824215_model_epoch_0006.pyth
[train_al.py: 579]: Wrote checkpoint to: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/auto_ml_results/lSet_1/start_1/CIFAR10/40.0/random/vgg_depth_16/vanilla/trial-4/checkpoints/vlBest_acc_50.979999694824215_model_epoch_0006.pyth
[I 2022-04-02 21:51:35,169] Trial 4 finished with value: 50.979999694824215 and parameters: {'learning_rate': 5.688271836020098e-05, 'weight_decay': 1.9330737818565223e-05, 'batch_size': 32, 'optimizer': 'SGD'}. Best is trial 2 with value: 56.71999847412109.
Time taken to finish study: 778.4632680416107 seconds
/nfs/users/ext_prateek.munjal/anaconda3/envs/pycls2/lib/python3.6/site-packages/optuna/structs.py:21: FutureWarning:

`structs` 

In [10]:
## Lets explore the results folder where all the transfer models are saved.

In [17]:
dirlist = []
for root, dirs, files in os.walk(out_dir+"/best_automl_results/"):
    for d in dirs:
        #append the dir name to the list
        if os.path.basename(d)=="checkpoints":
            dirlist.append(os.path.join(root,d))

dirlist.sort()            

#print all the dir names
print("-"*20)
print("Checkpoints dirs")
for d in dirlist:
    print("Found", d)


--------------------
Checkpoints dirs
Found /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/checkpoints
Found /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/20.0/random/vgg_depth_16/vanilla/checkpoints
Found /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/30.0/random/vgg_depth_16/vanilla/checkpoints
Found /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml_transfer/best_automl_results/lSet_1/start_1/CIFAR10/40.0/random/vgg_depth_16/vanilla/checkpoints


In [18]:
## We can also check the logged test accuracies for these transfer learning models.

In [51]:
def check_results(fpath, out_dir):
    fp = open(fpath, 'r')
    for line in fp.readlines():
        line=line.rstrip("\n")
        cur_path, cur_acc = line.split(" ")
        print("="*20)
        print(f"PATH: {cur_path.replace(out_dir, '')}") 
        print(f"TEST ACCURACY: {cur_acc}")
        print("="*20)
    fp.close()

In [53]:
#10% model results
fpath=f"{out_dir}/best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/test_acc_.txt"
check_results(fpath, out_dir)

#20% model results
fpath=f"{out_dir}/best_automl_results/lSet_1/start_1/CIFAR10/20.0/random/vgg_depth_16/vanilla/test_acc_.txt"
check_results(fpath, out_dir)

#30% model results
fpath=f"{out_dir}/best_automl_results/lSet_1/start_1/CIFAR10/30.0/random/vgg_depth_16/vanilla/test_acc_.txt"
check_results(fpath, out_dir)

#40% model results
fpath=f"{out_dir}/best_automl_results/lSet_1/start_1/CIFAR10/40.0/random/vgg_depth_16/vanilla/test_acc_.txt"
check_results(fpath, out_dir)

PATH: /best_automl_results/lSet_1/start_1/CIFAR10/10.0/vgg_depth_16/vanilla/checkpoints/vlBest_acc_47.3_model_epoch_0006.pyth
TEST ACCURACY: 48.020000228881834
PATH: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml/best_automl_results/lSet_1/start_1/CIFAR10/20.0/random/vgg_depth_16/vanilla/checkpoints/vlBest_acc_50.01999969482422_model_epoch_0006.pyth
TEST ACCURACY: 51.04999931335449
PATH: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml/best_automl_results/lSet_1/start_1/CIFAR10/30.0/random/vgg_depth_16/vanilla/checkpoints/vlBest_acc_65.63999855041504_model_epoch_0006.pyth
TEST ACCURACY: 65.24999839782714
PATH: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/sample_results_aml/best_automl_results/lSet_1/start_1/CIFAR10/40.0/random/vgg_depth_16/vanilla/checkpoints/vlBest_acc_64.77999839782714_model_epoch_0006.pyth
TEST ACCURACY: 65.47999843597412
